In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.model_selection import train_test_split

from gptchem.data import get_mof_solvent_data
from gptchem.formatter import MOFSolventRecommenderFormatter
from gptchem.querier import Querier
from gptchem.tuner import Tuner

In [3]:
data = get_mof_solvent_data()

In [4]:
solvent_columns = ["solvent1", "solvent2", "solvent3", "solvent4", "solvent5"]
solvent_molrations = [
    "sol_molratio1",
    "sol_molratio2",
    "sol_molratio3",
    "sol_molratio4",
    "sol_molratio5",
]
metal = ["core_All_Metals"]
counter_ions = ["counterions1"]
linkers = ["linker_1", "linker_2"]

In [5]:
recommender = MOFSolventRecommenderFormatter(
    linker_columns=["linker_1", "linker_2"],
    node_columns=["core_All_Metals"],
    counter_ion_columns=["counterions1"],
    solvent_columns=["solvent1", "solvent2", "solvent3", "solvent4", "solvent5"],
    solvent_mol_ratio_columns=[
        "sol_molratio1",
        "sol_molratio2",
        "sol_molratio3",
        "sol_molratio4",
        "sol_molratio5",
    ],
)

In [6]:
formatted = recommender(data)

In [7]:
formatted

,prompt,completion,label,representation,solvents,solvent_mol_ratios
0,In what solvent can one make a MOF out of [O-]...,0.53 CN(C)C=O and 0.18 C(CO)O and 0.28 O@@@,0.53 CN(C)C=O and 0.18 C(CO)O and 0.28 O,[[[O-]C(=O)c1cc([N][N]c2cc(cc(c2)C(=O)O)C(=O)O...,"[CN(C)C=O, C(CO)O, O, nan, nan]","[0.532, 0.184, 0.284, nan, nan]"
1,In what solvent can one make a MOF out of O=C(...,0.54 CN(C)C=O and 0.46 O@@@,0.54 CN(C)C=O and 0.46 O,[[O=C(c1cncc(c1)C(=O)Nc1ccc2c(c1)ccc(c2)C(=O)[...,"[CN(C)C=O, O, nan, nan, nan]","[0.539, 0.461, nan, nan, nan]"
2,In what solvent can one make a MOF out of [N]1...,1.0 O@@@,1.0 O,"[[[N]1C=NN=C1, nan], [Zn], [F], [O, nan, nan, ...","[O, nan, nan, nan, nan]","[1.0, nan, nan, nan, nan]"
3,In what solvent can one make a MOF out of [O-]...,0.56 CN(C)C=O and 0.34 O and 0.1 CCO@@@,0.56 CN(C)C=O and 0.34 O and 0.1 CCO,"[[[O-]C(=O)c1ccc(cc1)c1cccc(c1)C(=O)[O-], nan]...","[CN(C)C=O, O, CCO, nan, nan]","[0.556, 0.339, 0.105, nan, nan]"
4,In what solvent can one make a MOF out of [O-]...,0.38 CN(C)C=O and 0.3 CCO and 0.32 O@@@,0.38 CN(C)C=O and 0.3 CCO and 0.32 O,[[[O-]C(=O)c1cc([N][N]c2cc(cc(c2)C(=O)[O-])C(=...,"[CN(C)C=O, CCO, O, nan, nan]","[0.377, 0.3, 0.323, nan, nan]"
...,...,...,...,...,...,...
431,In what solvent can one make a MOF out of Cc1c...,1.0 CO@@@,1.0 CO,[[Cc1c(c2c(C)c(c3ccncc3)c(c(c2C)c2ccncc2)C)c(C...,"[CO, nan, nan, nan, nan]","[1.0, nan, nan, nan, nan]"
432,In what solvent can one make a MOF out of [O-]...,1.0 O@@@,1.0 O,[[[O-]C(=O)C1C(C(=O)[O-])C(C1C(=O)[O-])C(=O)[O...,"[O, nan, nan, nan, nan]","[1.0, nan, nan, nan, nan]"
433,In what solvent can one make a MOF out of [O]P...,1.0 O@@@,1.0 O,"[[[O]P(=O)(C(P(=O)([O])[O])(Cc1cccnc1)O)[O], n...","[O, nan, nan, nan, nan]","[1.0, nan, nan, nan, nan]"
434,In what solvent can one make a MOF out of [O-]...,1.0 O@@@,1.0 O,"[[[O-]C(=O)c1cccc(c1)C(=O)[O-], c1ccc(nc1)c1cc...","[O, nan, nan, nan, nan]","[1.0, nan, nan, nan, nan]"


In [8]:
train, test = train_test_split(formatted, train_size=100, test_size=50, random_state=42)

In [10]:
tuner = Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
tuner(train)

KeyboardInterrupt: 

In [ ]:
querier = Querier.from_preset(tune_res["model_name"])
completions = querier(test)

NameError: name 'tune_res' is not defined

In [11]:
import re

In [12]:
s = "0.53 CN(C)C=O and 0.18 C(CO)O and 0.28 O@@@"

In [17]:
find_solvents = re.compile(r"(\d+\.\d+)(\s[\w\(\)=\@]+)")

In [18]:
find_solvents.findall(s)

[('0.53', ' CN(C)C=O'), ('0.18', ' C(CO)O'), ('0.28', ' O@@@')]